In [ ]:
"""Image pre-processing related to binarisation"""

import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

version = 10
specimen = 1

color_space = "Hsv"
ppm = 25
image_path = f"/Users/rashid/Library/CloudStorage/OneDrive-UniversityofBristol/Year 3/University Work/Term 2/IRP/4. Decoding Information/Colored_QR_Decoding_Full/Full Pictures/25ppm_v{version}_Hsv_Scan.jpg"

img = cv.imread(image_path, 1)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(gray_img)
blur = cv.GaussianBlur(cl1,(5,5),0)
_,binarized_image = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

plt.figure(figsize=(10, 10), dpi = 300)
plt.imshow(binarized_image, cmap="gray")
plt.axis("off")
plt.show()


In [ ]:
"""Image segmentation of the Colored Qr Code"""
from matplotlib import pyplot as plt
import numpy as np
from scipy.spatial import distance as dist
from colorsys import hsv_to_rgb, rgb_to_hsv
from transformers import pipeline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps


def image_segmentation(image_path):
    pipe = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True)
    mask = pipe(image_path, return_mask = True) 
    image_pillow = pipe(image_path)
    temp_image_path = "temp.png"
    image_pillow.save(temp_image_path)
    image = cv.imread(temp_image_path)
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    return image_rgb, mask

image, mask = image_segmentation(image_path)
plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
""""Binarised image of the segmented image"""
gray_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

# create a CLAHE object (Arguments are optional).
clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(gray_img)

blur = cv.GaussianBlur(cl1,(21,21),0)
_,binarized_image = cv.threshold(cl1,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
plt.imshow(binarized_image, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
"""Image edge detection"""

def CANNY_ED(img, sigma = 0.3):
    blur = cv.GaussianBlur(img, (5,5), cv.BORDER_CONSTANT)
    median = np.median(img)
    lower = int(max(0, (1.0-sigma)*median))
    upper = int(min(255, (1.0+sigma)*median))
    autocanny = cv.Canny(blur, lower, upper)
    return autocanny

canny = CANNY_ED(binarized_image)
plt.imshow(canny, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
"""Dilation of the canny edges to make them more distinct"""
kernel = np.ones((5, 5), np.uint8) 

img_dilation = cv.dilate(canny, kernel, iterations=5)

plt.imshow(img_dilation, cmap="summer")
plt.axis('off')
plt.show()

In [ ]:
"""Finding the contour of the box and then segmenting the image and cropping it"""
contours_dilate, _ = cv.findContours(img_dilation,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE )

contours_sorted = sorted(contours_dilate, key=cv.contourArea, reverse=True)

c = contours_sorted[0]
rect = cv.minAreaRect(c)
box = cv.boxPoints(rect)
box = np.array(box, dtype="int")

ordered_box = np.zeros((4, 2), dtype="float32")
s = box.sum(axis=1)
ordered_box[0] = box[np.argmin(s)]
ordered_box[2] = box[np.argmax(s)]
diff = np.diff(box, axis=1)
ordered_box[1] = box[np.argmin(diff)]
ordered_box[3] = box[np.argmax(diff)]

side = max([np.linalg.norm(ordered_box[i] - ordered_box[(i + 1) % 4]) for i in range(4)])
dst_pts = np.array([[0, 0], [side - 1, 0], [side - 1, side - 1], [0, side - 1]], dtype="float32")

M = cv.getPerspectiveTransform(ordered_box, dst_pts)
warped = cv.warpPerspective(img, M, (int(side), int(side)))

plt.imshow(warped)
plt.axis('off')
plt.show()

In [ ]:
"""Binarising the cropped image"""
gray_img1 = cv.cvtColor(warped, cv.COLOR_BGR2GRAY)

# create a CLAHE object (Arguments are optional).
clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(gray_img1)

blur = cv.GaussianBlur(cl1,(5, 5),0)
_,binarized_image = cv.threshold(cl1,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

In [ ]:
"""Finding the corners"""
height, width = binarized_image.shape
coordinates = []
for y in range(height):
    for x in range(width):
        if binarized_image[y, x] == 0:
            coordinates.append((x, y))

ordered_box1 = np.zeros((4, 2), dtype="float32")
ordered_box1[2] = max(coordinates, key=lambda coord: coord[0] + coord[1])
ordered_box1[0] = min(coordinates, key=lambda coord: coord[0] + coord[1])
ordered_box1[1] = max(coordinates, key=lambda coord: coord[0] - coord[1])
ordered_box1[3] = min(coordinates, key=lambda coord: coord[0] - coord[1])

plt.imshow(warped)
plt.scatter(ordered_box1[0][0], ordered_box1[0][1], color='yellow', marker='o', s=10)  # Plot top left corner
plt.scatter(ordered_box1[1][0], ordered_box1[1][1], color='yellow', marker='o', s=10)  # Plot bottom right corner
plt.scatter(ordered_box1[2][0], ordered_box1[2][1],color='yellow', marker='o', s=10)  # Plot top right corner
plt.scatter(ordered_box1[3][0], ordered_box1[3][1], color='yellow', marker='o', s=10) 

plt.axis('off')
plt.show()

In [ ]:
"""Perspective and affine transform to get the final fixed box"""
#side1 = max([np.linalg.norm(ordered_box1[i] - ordered_box1[(i + 1) % 4]) for i in range(4)])
side1 = (version*4 + 17)*50
#Poitns where the box will be mapped onto
dst_pts1 = np.array([[0, 0], [side1 - 1, 0], [side1 - 1, side1 - 1], [0, side1 - 1]], dtype="float32")

# Warp the image using the perspective matrix
M1 = cv.getPerspectiveTransform(ordered_box1, dst_pts1)
warped2 = cv.warpPerspective(warped, M1, (int(side1), int(side1)))

plt.imshow(warped2, cmap="gray")
plt.axis("off")
plt.show()


In [ ]:
"""White balancing"""

from skimage.util import img_as_ubyte
def percentile_whitebalance(image, percentile_value = 90):
        fig, ax = plt.subplots(1,2, figsize=(12,6))
        for channel, color in enumerate('rgb'):
            channel_values = image[:,:,channel]
            value = np.percentile(channel_values, percentile_value)
            ax[0].step(np.arange(256), 
                    np.bincount(channel_values.flatten(), 
                    minlength=256)*1.0 / channel_values.size, 
                    c=color)
            ax[0].set_xlim(0, 255)
            ax[0].axvline(value, ls='--', c=color)
            ax[0].text(value-70, .01+.012*channel, 
                    "{}_max_value = {}".format(color, value), 
                        weight='bold', fontsize=10)
            ax[0].set_xlabel('channel value')
            ax[0].set_ylabel('fraction of pixels')
            ax[0].set_title('Histogram of colors in RGB channels')    
            whitebalanced = img_as_ubyte(
                    (image*1.0 / np.percentile(image, 
                    percentile_value, axis=(0, 1))).clip(0, 1))
            ax[1].imshow(whitebalanced);
            ax[1].axis('off')
            ax[1].set_title('Whitebalanced Image')
        return ax, whitebalanced
    
ax, wb = percentile_whitebalance(warped2, 99)

In [ ]:
"""Finding the dominant color in each module using k means clustering with a single centroid only. The function only"""

def find_dominant_color(image_np, k=1):
    # Ensure the image is in the correct shape and not empty
    if image_np.size == 0 or image_np.ndim != 3 or image_np.shape[2] != 3:
        return (0, 0, 0)  # Return a default color if there's no data to process
    
    # Convert image to the correct color scheme (OpenCV uses BGR)
    image_np = cv.cvtColor(image_np, cv.COLOR_RGB2BGR)
    
    # Reshape the image to be a list of pixels
    pixels = image_np.reshape((-1, 3))
    
    # Handle case where there are fewer pixels than clusters
    if pixels.shape[0] < k:
        return tuple(map(int, np.mean(pixels, axis=0)))  # Return the mean color if too few pixels for k-means

    # Convert to floating point for OpenCV kmeans function
    pixels = np.float32(pixels)
    
    # Define criteria and apply kmeans()
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centroids = cv.kmeans(pixels, k, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)
    
    # Convert centroids to int and get the dominant color
    dominant_color = centroids[0].astype(int)
    
    # Convert the dominant color back to RGB color space
    dominant_color = tuple(int(c) for c in dominant_color[::-1])
    
    return dominant_color

def find_average_color(image):
    mean = np.mean(image, axis=(0,1))
    std_dev = np.std(image, axis=(0,1))
    return mean, std_dev

# Assuming 'image_array' is your image as a numpy array

In [ ]:
n = wb
height, width, _ = wb.shape
num_parts = version*4 + 17
box_height = box_width = 50

average_colors = []
std_devs = []
for i in range(num_parts):
    start_y = i * box_height
    end_y = min((i + 1) * box_height, height)

    for j in range(num_parts):
        start_x = j * box_width
        end_x = min((j + 1) * box_width, width)

        # Extract the image section directly
        box = n[start_y:end_y, start_x:end_x]

        # Compute the dominant color of the section
        dominant_color = find_dominant_color(box)

        average_colors.append(dominant_color)


# Convert the list to a NumPy array for further processing
average_colors = np.array(average_colors)
#std_devs = np.array(std_dev)

print(average_colors.shape)

def Color_Matrix_with_Border(array, w, h, ppm, bw=0, bc=(0,0,0)):
    obw = 2 * bw
    mw = ppm + bw
    tw = w * mw + 2 * obw  - bw
    th = h * mw + 2 * obw  - bw
    
    data = np.zeros((th, tw, 3), dtype=np.uint8)
    data[:] = bc

    for i, array in enumerate(array):
        row = (i // w) * mw + obw
        col = (i % w) * mw + obw
        data[row:row+ppm, col:col+ppm] = array
    
    return data

w, h = int(np.sqrt(len(average_colors))), int(np.sqrt(len(average_colors)))
data = Color_Matrix_with_Border(average_colors, w, h, 1)
img1 = np.array(Image.fromarray(data, 'RGB'))
plt.figure(figsize=(10, 10), dpi=300)
plt.imshow(img1)
plt.axis('off')
plt.show()

In [ ]:
"""Getting the color reference box from the colored qr code at the bottom right corner"""
width = height = int(np.sqrt(len(average_colors)))
#ref_col = average_colors.reshape((width,height, 3))
col_ref = img1[width - 17:width -1, height - 17: height-1] #ref_box
colors_det = col_ref.reshape((256, 3)) #ref_colors
array1 = colors_det

"""plt.imshow(col_ref)
plt.axis('off')
plt.show()"""

In [ ]:
from CQRCode import CQrCode
#file_path = "/Users/rashid/Library/CloudStorage/OneDrive-UniversityofBristol/Year 3/University Work/Term 2/IRP/3. Encoding Information/Colored_QrCode_Encoding/sample4.aac"

file_path = r"/Users/rashid/Library/CloudStorage/OneDrive-UniversityofBristol/Year 3/University Work/Term 2/IRP/3. Encoding Information/Colored_QrCode_Encoding/sample4.aac"

cqr = CQrCode(version, "hsv", file_path)
matrix = cqr._isfunction

modules = 0
positions = []
for i in range(len(matrix)):
    for j in range(len(matrix)):
        if matrix[i][j] == 255:
            modules += 1
            positions.append((i,j))
        else:
            pass
print(img1.shape)
print()

In [ ]:

data_colors = []
for i in positions:
    data_colors.append(img1[i[0]][i[1]])
    print(img1[i[0]][i[1]])

data_colors = np.array([list(i) for i in data_colors])


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import color
from skimage.color import deltaE_ciede2000

def delta_e_distance(c1, c2):
    return deltaE_ciede2000(c1, c2)

def quantize_using_delta_e(img_array, palette):
    # Convert RGB image array to CIELAB
    lab_image = color.rgb2lab(img_array)
    # Convert palette to CIELAB for comparison
    lab_palette = color.rgb2lab(palette.reshape(-1, 1, 3))
    
    # Flatten the LAB image to a 2D array of colors
    pixel_array = lab_image.reshape((-1, 1, 3))
    
    # Prepare array to receive quantized colors' indices
    quantized_indices = np.zeros(len(pixel_array), dtype=int)
    
    # Find the index of the closest color in the LAB palette for each pixel
    for i in range(len(pixel_array)):
        distances = [delta_e_distance(pixel_array[i], lab_color) for lab_color in lab_palette]
        quantized_indices[i] = np.argmin(distances)
    
    # Use the indices to map directly back to the RGB palette, ensuring exact color match
    quantized_array = palette[quantized_indices]
    
    # Reshape the quantized array back to the original image shape
    return quantized_array.reshape(img_array.shape)


new_colors = np.array([[0, 0, 0], [255, 255, 255]])
palette = np.concatenate((array1, new_colors))
quantized_data = quantize_using_delta_e(data_colors, np.array(array1))


In [ ]:
array1 = [[i[0], i[1], i[2]] for i in list(array1)]
print(array1)

In [ ]:
data_colors = [[i[0], i[1], i[2]] for i in list(quantized_data)]
data_decimal = []


for i in data_colors:
    decimal = array1.index(i)
    data_decimal.append(decimal)

print(data_decimal)



In [ ]:
original_data = cqr._read_data(file_path)

original_data = original_data[:len(data_decimal)]

print(original_data)

In [ ]:
matched = 0

for i in range(len(original_data)):
    if original_data[i] == data_decimal[i]:
        matched += 1
    else:
        pass

success = (matched/len(original_data))*100
print(f"Success Rate of Perfect Matches: {success:.2f}%")

In [ ]:
#Need this one
versions = [3, 10, 20, 40, 60, 90]
success_rates_avg = [17.56, 17.96, 12.12, 11.1, 8.14, 1.32]

# Plotting
plt.figure(figsize=(15, 5))  # Set a suitable figure size
plt.plot(versions, success_rates_avg, marker='o', linestyle='-', color='b')  # Line plot

# Add a scatter plot overlay if you want each point to stand out
plt.scatter(versions, success_rates_avg, color='r')  # Add emphasis to each data point

# Formatting plot
plt.xlabel('Version', fontsize=14)  # X-axis label
plt.ylabel('Perfect Match Success Rate (%)', fontsize=14)  # Y-axis label
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)  # Enable grid for better readability
plt.xticks(versions)  # Ensure all versions are marked
plt.ylim(0, max(success_rates_avg) * 1.1)  # Set y-axis limits to show all data clearly
plt.xlim(min(versions))

# Legend
plt.legend()

# Show plot
plt.tight_layout()  # Adjust layout to make sure everything fits without overlap
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming 'data_decimal' and 'original_data' are defined

plt.figure(figsize=(25, 10))  # Adjust figure size for better proportionality
plt.plot(data_decimal, label='Detected Values', color='blue', linestyle='-', marker='o', markersize=5, linewidth=2)
plt.plot(original_data, label='True Values', color='darkgray', linestyle="--", marker='x', markersize=5, linewidth=2)

# Adding labels and legend
plt.xlabel('Data Module Index', fontsize=14)
plt.ylabel('Values', fontsize=14)
plt.xlim(0, len(data_decimal) - 1)  # Set proper x-axis limits
plt.ylim(min(min(data_decimal), min(original_data)) * 0.95, max(max(data_decimal), max(original_data)) * 1.05)  # Dynamic y-limits

# Improve tick visibility and count
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Adding a grid
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7, color='lightgrey')

# Legend with better placement to avoid blocking the data
plt.legend(fontsize=14, loc='upper left', frameon=True, facecolor='whitesmoke', framealpha=0.8, edgecolor='black')

# Title for context

# Display the plot
plt.tight_layout()  # Adjust layout to make sure everything fits without overlap
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_rrmse(predicted, true):
    """Calculate the Relative Root Mean Squared Error between predicted and true values."""
    if true == 0:
        return None  # Avoid division by zero and return None for this case
    errors = predicted - true
    squared_relative_errors = (errors / true) ** 2
    mean_squared_relative_errors = np.mean(squared_relative_errors)
    rrmse = np.sqrt(mean_squared_relative_errors)
    return rrmse

def calculate_rmse(predicted, true):
    """Calculate the Root Mean Squared Error between predicted and true values."""
    errors = predicted - true
    squared_errors = np.square(errors)  # More explicit and numpy vectorized operation
    mean_squared_errors = np.mean(squared_errors)
    rmse = np.sqrt(mean_squared_errors)
    return rmse

# Assuming data_decimal and original_data are defined as numpy arrays for vectorized operations
# Calculate RRMSE for each module
rrmse_modules = [calculate_rrmse(pred, true) if true != 0 else 0 for pred, true in zip(data_decimal, original_data)]

# Plot the RRMSE for each module
plt.figure(figsize=(30, 10))
plt.plot(rrmse_modules, linestyle='-', color='b')  # Adding markers for better visibility
plt.xlabel('Data Module Index', fontsize=14)
plt.ylabel('RRMSE (%)', fontsize=14)
plt.xlim(0, len(rrmse_modules) - 1)
plt.ylim(0, max(rrmse_modules) * 1.1)  # Give some space above the highest value
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
color_grid = np.array([row[:] for row in img1])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# Assume data_decimal, original_data, color_grid, and positions are already defined

def calculate_rrmse(predicted, true):
    """Calculate the Relative Root Mean Squared Error between predicted and true values."""
    if true == 0:
        return None  # Return None if true value is zero to avoid division by zero
    errors = predicted - true
    squared_relative_errors = (errors / true) ** 2
    mean_squared_relative_errors = np.mean(squared_relative_errors)
    rrmse = np.sqrt(mean_squared_relative_errors)
    return rrmse

# Calculate RRMSE values, including None for true values of zero

color_grid = np.array([row[:] for row in img1])
rrmse_modules = [calculate_rrmse(pred, true) if true != 0 else None for pred, true in zip(data_decimal, original_data)]

# Dimensions of the grid
w_parts, h_parts, _ = color_grid.shape

# Create an overlay grid for errors, initializing with NaNs (which represent no error)
error_grid = np.full((w_parts, h_parts), np.nan)

# Fill the error grid with error values at specified positions
for pos, err in zip(positions, rrmse_modules):
    if err is not None:
        error_grid[pos] = err

# Plotting
fig, ax = plt.subplots(figsize=(10, 10), dpi=600)

# Plot color grid as background
ax.imshow(color_grid, origin='upper', interpolation='nearest', extent=(0, w_parts, h_parts, 0))

# Overlay heatmap for errors
norm = Normalize(vmin=np.nanmin(error_grid[np.isnan(error_grid) == False]), vmax=np.nanmax(error_grid))
cmap = plt.get_cmap('hot')

# Create a color mask for error values
for (j, i), value in np.ndenumerate(error_grid):
    if not np.isnan(value):
        color = cmap(norm(value))
        ax.add_patch(plt.Rectangle((i, j), 1, 1, color=color))

# Formatting plot
ax.set_xlim(0, w_parts)
ax.set_ylim(h_parts, 0)
ax.grid(False)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.axis('off')

# Add a colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
"""cbar = plt.colorbar(sm, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('RRMSE Error Magnitude')"""

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Given data
versions = [3, 10, 20, 40, 60, 90]
success_rates_avg = [17.56, 17.96, 12.12, 11.1, 8.14, 1.32]
modules_list = [262, 2499, 8414, 29379, 62694, 136174]  # Also the bytes of data
bitrate_standard = 64  # kbps
bitrate = 64 * 1000 / 8  # B/s

# Calculate seconds for each modules value
seconds = [i / bitrate for i in modules_list]

# Plotting
fig, ax1 = plt.subplots(figsize=(15, 5))  # Set a suitable figure size

# Line plot for Success Rates
ax1.plot(versions, success_rates_avg, 'b-o', label='Average Success Rate (%)')  # Blue line
ax1.set_xlabel('Version', fontsize=14)
ax1.set_ylabel('Success Rate (%)', fontsize=14, color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_xticks(versions)  # Ensure all versions are marked
ax1.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)

# Set x-axis limits
ax1.set_xlim([min(versions) - 5, max(versions) + 5])  # Extend limits a bit beyond the min and max values for clarity

# Create a secondary y-axis for seconds
ax2 = ax1.twinx()
ax2.plot(versions[:len(seconds)], seconds, 'r-s', label='Storage Capacity (s)')  # Red line
ax2.set_ylabel('Storage Capacity (seconds)', fontsize=12, color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper center')

plt.tight_layout()  # Adjust layout to make sure everything fits without overlap
plt.show()

In [ ]:

# Calculate modules_detected
modules_detected = [int(success_rates_avg[i] * modules_list[i] / 100) for i in range(len(success_rates_avg))]

# Calculate rate of change of modules_detected
rate_of_change = np.diff(modules_detected) / np.diff(versions)

# Adjust versions for rate of change calculation (mid-points of versions)
versions_diff = [(versions[i] + versions[i + 1]) / 2 for i in range(len(versions) - 1)]

# Plotting
fig, ax1 = plt.subplots(figsize=(15, 5))

# Line plot for modules_detected
ax1.plot(versions, modules_detected, 'b-o', label='Modules Detected')
ax1.set_xlabel('Version', fontsize=14)
ax1.set_ylabel('Modules Detected', fontsize=14, color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)

# Create secondary y-axis for rate of change
ax2 = ax1.twinx()
ax2.plot(versions_diff, rate_of_change, 'r-s', label='Rate of Change of Modules Detected')
ax2.set_ylabel('Rate of Change', fontsize=15, color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper right')

plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
def percentage_below_threshold(items, threshold):
    count_below_threshold = sum(1 for item in items if item < threshold)
    
    # Total number of items
    total_items = len(items)
    
    if total_items == 0:
        return 0 
    
    # Calculate the percentage
    percentage = (count_below_threshold / total_items) * 100
    
    return percentage

rrmse_modules = [r for r in rrmse_modules if r is not None]
results =  percentage_below_threshold(rrmse_modules, 1)
print(f"Successful Results Below Threshold: {results:.2f}%")

In [ ]:
def writing_file(array, version):
    destination_path = r"C:\Users\rr21134\OneDrive - University of Bristol\Year 3\University Work\Term 2\IRP\4. Decoding Information\Colored_QR_Decoding_Full\90test.aac"
    with open(destination_path, 'wb') as file:
        file.write(bytes(array))

writing_file(data_decimal, 90)